## 1. Production planning problem

$max \ \ -40x+130y_f+60z_f+90y_h+40z_h$

Subject to  
$$x \le 20 \\
y_f+\frac{3}{4}y_h \le 8 \\
y_h+z_h = 0.5x \\
y_f+z_f = 0.5x \\
x,y,z >=0$$

In [ ]:
import numpy as np
import cvxpy as cp
import scipy


In [ ]:
x = cp.Variable(1)
y = cp.Variable(2)
z = cp.Variable(2)

In [ ]:
obj = cp.Maximize(-40*x[0]+130*y[0]+60*z[0]+90*y[1]+40*z[1])

In [ ]:
constraints = [
               x[0] <= 20,
               x[0] >=0,
               y>=0,
               z>=0,
               y[0]+0.75*y[1] <= 8,
               y[1]+z[1] == 0.5*x[0],
               y[0]+z[0] == 0.5*x[0]
]
prob = cp.Problem(obj,constraints)
prob.solve(solver = cp.GLPK)
if (prob.status == 'optimal'):
  print('value = ' + str(prob.value))
  print('number of 1000 barrels not processed: z = ' + str(z.value))
  print('number of 1000 barrels processed: y = ' + str(y.value))


value = 760.0
number of 1000 barrels not processed: z = [ 2. 10.]
number of 1000 barrels processed: y = [8. 0.]


## 2. Stock liquidation problem


In [ ]:
shares_sold = cp.Variable(10)
shares_remained = 100-shares_sold
price_future = np.array([36,39,42,45,51,55,63,64,66,70])
price_current = np.array([30,34,43,47,49,53,60,62,64,66])
price_purchase = np.array([20,25,30,35,40,45,50,55,60,65])
obj = cp.Maximize(shares_remained*price_future)
constraints = [
               shares_sold*(0.69*price_current+0.3*price_purchase)==30000,
               shares_sold <=100,
               shares_sold >=0
]
prob = cp.Problem(obj,constraints)
prob.solve(solver = cp.GLPK)
if (prob.status == 'optimal'):
  print('Maximized expected return = ' + str(prob.value))
  print('shares sold ',np.around(shares_sold.value,3))


Maximized expected return = 20893.708807669267
shares sold  [  0.      0.    100.    100.     -0.     63.751   0.    100.    100.
 100.   ]


## 3. Refinery planning problem
• xi = number of barrels of crude processed using method i = 1, 2, 3. \\
• $y_{rg}$ = number of barrels of grade r crude converted into gas \\
• $y_{rh} $= number of barrels of grade r crude converted into heating oil\\ 

• z1 = number of barrels of grade 6 oil cracked into grade 8 \\
• z2 = number of barrels of grade 8 oil cracked into grade 10\\

In [ ]:
x = cp.Variable(3)
y_rg = cp.Variable(3)
y_rh = cp.Variable(3)
z = cp.Variable(2)
cost_methods = np.array([3.4,3,2.6])

profit = []
for i in range(3):
    profit += [y_rh[i]*5+y_rg[i]*12-x[i]*cost_methods[i]]

grade = np.array([6,8,10])
obj = cp.Maximize(sum(profit)-z[0]-1.5*z[1])


In [ ]:
constraints = [
               y_rh[0]+y_rh[1]+y_rh[2]<=600,
               sum(y_rg)<=2000,
               np.mean(y_rh*grade) >= 7,
               np.mean(y_rg*grade) >= 9,
               0.3*x[0]+0.4*x[1]+0.1*x[2] == z[0]+y_rg[0]+y_rh[0],     # inflow of different grades crude = outflow
               0.5*x[0]+0.2*x[1]+0.3*x[2]+z[0] == z[1]+y_rg[1]+y_rh[1],
               0.8*x[0]+0.4*x[1]+0.2*x[2]+z[1] == y_rg[2]+y_rh[2],
               x>=0,y_rh>=0,y_rg>=0,z>=0
]

In [ ]:
prob = cp.Problem(obj,constraints)
prob.solve(solver = cp.GLPK)
if (prob.status == 'optimal'):
  print('Maximized profit = ' + str(prob.value))
  print('Number of barrels of crude processed using different methods ' + str(x.value))
  print('Number of barrels converted into gas: ',y_rg.value)
  print('Number of barrels converted into heating oil: ',y_rh.value)
  print('Number of barrels of grade 6,8 cracked',z.value)


Maximized profit = 21475.000000000007
Number of barrels of crude processed using different methods [1625.    0.    0.]
Number of barrels converted into gas:  [   0.   812.5 1187.5]
Number of barrels converted into heating oil:  [ 4.87500000e+02 -1.13686838e-13  1.12500000e+02]
Number of barrels of grade 6,8 cracked [0. 0.]


In [ ]:
import pandas as pd
pd.DataFrame({'Number of barresls':x.value},index=['method 1','method 2','method 3'])

,Number of barresls
method 1,1625.0
method 2,0.0
method 3,0.0


In [ ]:
pd.DataFrame({'y_rg':y_rg.value,'y_rh':np.around(y_rh.value,2),'z':np.append(z.value,'NA')},index=['grade6','grade 8','grade 10'])

,y_rg,y_rh,z
grade6,0.0,487.5,0.0
grade 8,812.5,-0.0,0.0
grade 10,1187.5,112.5,NA
